# Evaluating Numerical Expressions

In this project, we will use stacks to implement an algorithm that can evaluate numerical expressions.

First we will import a class from another file and then create a Stack class.

In [1]:
from linked_list import LinkedList

class Stack(LinkedList):
    
    def push(self, data):
        self.append(data)

    def peek(self):
        return self.tail.data

    def pop(self):
        ret = self.tail.data
        if self.length == 1:
            self.tail = self.head = None
        else:
            self.tail = self.tail.prev
            self.tail.next = None
        self.length -= 1
        return ret

## Implementing the Tokenize Function - Infix vs Postfix Notation

When we write an expression, we use infix notation, meaning that we put the operators between the two operands. For example 1 + 2 is in infix notation because the + operator is between the operands 1 and 2.

For a computer, it's much easier to evaluate an expression written in postfix notation. In postfix notation, the operands appear before the operator. The infix expression 1 + 2 becomes 1 2 + in postfix notation.

In the context of evaluating expressions, we call these elements tokens, and the term for transforming the expression into a list of tokens is tokenize.

We will do that below.

In [2]:
def tokenize(expression):
    return expression.split()

print(tokenize("12 2 4 + / 21 *"))

['12', '2', '4', '+', '/', '21', '*']


# Processing an Operator

1. If we find a number, then we push that number to the top of the stack.
2. If we find an operator, we pop the top two elements of the stack, perform the operation, and then push back the result.

We need to handle processing the operators which we will do below:

In [3]:
def process_minus(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top - top
    stack.push(result)
    
def process_plus(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    # Same as process_minus but with + instead of -
    result = second_to_top + top
    stack.push(result)
    
def process_times(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    # Same as process_minus but with * instead of -
    result = second_to_top * top
    stack.push(result)

def process_divide(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    # Same as process_minus but with / instead of -
    result = second_to_top / top
    stack.push(result)
    
def process_pow(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    # Same as process_minus but with ** instead of -
    result = second_to_top ** top
    stack.push(result)

## Evaluating Postfix Expressions

We will do the following for evaluating postfix expressions:

1. Tokenize the expression using the tokenize() function
2. Initialize an empty stack
3. For each token, do the following:
    * If the token is an operator, call the corresponding function to process it. For example, if we find a +, we call the process_plus() function.
    * Otherwise (the token is a number) we push that number to the top of the stack. Since each token is a string, we'll need to convert it to a float first.
4. Return the value that is left in the stack.

In [4]:
def evaluate_postfix(expression):
    tokens = tokenize(expression)
    stack = Stack()
    for token in tokens:
        if token == "+":
            process_plus(stack)
        elif token == "-":
            process_minus(stack)
        elif token == "*":
            process_times(stack)
        elif token == "/":
            process_divide(stack)
        elif token == "**":
            process_pow(stack)
        else:
            # The token is not an operator so it must be a number
            stack.push(float(token))
    return stack.pop()

In [5]:
expressions = [
    "4 6 -",
    "4 1 2 9 3 / * + 5 - *",
    "1 2 + 3 -",
    "1 2 - 3 +",
    "10 3 5 * 16 4 - / +",
    "5 3 4 2 - ** *",
    "12 2 4 + / 21 *",
    "1 1 + 2 **",
    "1 1 2 ** +"
]

for exp in expressions:
    print(evaluate_postfix(exp))

-2.0
8.0
0.0
2.0
11.25
45.0
42.0
4.0
2.0


## Operator Precedence in Infix Notation

To convert an expression from infix to postfix, we'll implement the Shunting-yard algorithm. The data structure to implement this algorithm is (again) a stack.

In [6]:
precedence = {
    "+": 1,
    "-": 1,
    "*": 2,
    "/": 2,
    "**": 3
}

print(precedence["/"] < precedence["-"])
print(precedence["+"] < precedence["*"])
print(precedence["+"] < precedence["-"])
print(precedence["/"] < precedence["**"])

False
True
False
True


## Processing Tokens in Infix to Postfix

Here's how processing should go for each token:

* Opening parenthesis, (:
 * Push the token into the stack for later use when we find a closing parenthesis.
* Closing parenthesis ):
 * While the top of the stack isn't an opening parenthesis, (, pop the top element, and append it to the postfix token list.
 * Pop the opening parentheses out of the stack at the end.
* Operator, +, -, *, /, or **:
 * While the top of the stack is also an operator with a precedence greater than or equal to this operator, pop the top element, and append it to the postfix token list.
 * Push the current operator to the top of the stack
* Operand (any number):
 * Append the number to the postfix token list.

In [7]:
def process_opening_parenthesis(stack):
    stack.push("(")
    
def process_closing_parenthesis(stack, postfix):
    # Add tokens until we find the open bracket
    while stack.peek() != "(":
        postfix.append(stack.pop())
    # Remove the opening bracket
    stack.pop()
    
def process_operator(stack, postfix, operator):
    while len(stack) > 0 and stack.peek() in precedence and precedence[stack.peek()] >= precedence[operator]:
        postfix.append(stack.pop())
    stack.push(operator)
    
def process_number(postfix, number):
    postfix.append(number)

## Implementing the Shunting-yard Algorithm

This function will work as follows:

1. We start by splitting the expression into tokens using the tokenize() function.
2. We initialize an empty stack.
3. We initialize an empty postfix token list.
4. Iterate over all tokens, and for each, do the following:
 * If the token is "(", we call the process_opening_parenthesis() function.
 * If the token is ")", we call the process_closing_parenthesis() function.
 * If the token is an operator, we call the process_operator() function.
 * Otherwise, the token is a number, and we call the process_number() function.
5. After processing all tokens, we use a while loop to pop the remaining stack element into the postfix token list.
6. Use the str.join() method to convert the postfix token list into a string.

In [8]:
def infix_to_postfix(expression):
    tokens = tokenize(expression)
    stack = Stack()
    postfix = []
    for token in tokens:
        if token == "(":
            process_opening_parenthesis(stack)
        elif token == ")":
            process_closing_parenthesis(stack, postfix)
        elif token in precedence:
            process_operator(stack, postfix, token)
        else:
            process_number(postfix, token)
    while len(stack) > 0:
        postfix.append(stack.pop())
    return " ".join(postfix)

## Evaluating Infix Expre